In [1]:
from keras.models import Sequential
from keras.layers import Input,Conv2D,Lambda,LeakyReLU,Activation,Concatenate,MaxPooling2D,BatchNormalization,Flatten,Dense,Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from keras.applications.densenet import DenseNet121
#import tensorflow as tf
from keras.models import Model
from tensorflow.keras import metrics
import numpy as np #矩阵库
import cv2         #图像处理库
import matplotlib #绘图库
import pandas as pd      #数据处理库
import os
import sys
import random
from datetime import datetime
from subprocess import check_output

#设置全局变量
ImageNameDataHash={}
uniquePatientIDList = []
def loadData(trainDir): #读取图像数据及处理
    global ImageNameDataHash
    
    images=os.listdir(trainDir)
    print("图片共有"+str((len(images)))+"张")
    
    for imageFileName in images:
        imageFullPath=os.path.join(trainDir,imageFileName)
        #print(imageFullPath)
        img=load_img(imageFullPath)
        #print(img)
        arr=img_to_array(img)
        #print(arr.shape)
        dim0=arr.shape[0]#高度
        dim1=arr.shape[1]#宽度
        dim2=arr.shape[2]#深度
        #调整图片尺寸统一
        if (dim0<256 or dim1<256 or dim2<3):
            print("图片错误"+str(arr.shape))
        arr=cv2.resize(arr,(128,128))
        dim0=arr.shape[0]#新高度
        dim1=arr.shape[1]#新宽度
        dim2=arr.shape[2]#新深度
        #print(new_dim2)
        #确定修改完成
        if (dim0!=128 or dim1!=128 or dim2!=3):
            print("图片错误"+str(arr.shape))
        #print(arr)
        #提取各颜色空间并生成LGI颜色空间
        #Blue=arr[:,:,0]
        #Green=arr[:,:,1]
        #Red=arr[:,:,2]
        
        #Lab=cv2.cvtColor(arr,cv2.COLOR_BGR2Lab)
        #Light=Lab[:,:,0]
        
        #I1=(Blue+Green+Red)/3
        
        #arr=cv2.merge([Green,Light,I1])
        #print(new_arr.shape)
        #arr=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
        #arr=np.array(arr,dtype="float")#归一化
        imageFileName = imageFileName.replace('.jpg','')
        #print(imageFileName)
        ImageNameDataHash[str(imageFileName)] = np.array(arr)
        #print(ImageNameDataHash)
    print('完成')
    return

print("读取数据"+str(datetime.now()))
sys.stdout.flush()
loadData("D:/BaiduNetdiskDownload\data/B. Disease Grading/B. Disease Grading/1. Original Images/a. Training Set")



Using TensorFlow backend.


读取数据2020-03-10 20:59:43.761741
图片共有413张
完成


In [2]:
def loadLabels():#读取标签
    raw_df=pd.read_csv('a. IDRiD_Disease Grading_Training Labels.csv',sep=',',usecols=[0,1])
    #print(type(raw_df))
    #raw_df["PatientID"]=''#添加病人ID
    #print("共有"+str(raw_df.shape[0])+"数据\n"+"拥有"+str(raw_df.shape[1])+"特征")
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    #print(row_count)
    #print(col_count)
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    ImageLevelHash = {}
    #patientIDList = []
    for index, row in raw_df.iterrows():
        key = row[0] + ''
        #patientID = row[0] + ''
        #patientID = patientID.replace('_right','')
        #patientID = patientID.replace('_left','')#删除标签，从而设置病人ID
        #raw_df.at[index, 'PatientID'] = patientID
        #patientIDList.append(patientID)
        ImageLevelHash[key] = str(row[1]) 
        
    #print(ImageLevelHash)
    
    #global uniquePatientIDList
    #uniquePatientIDList=sorted(set(patientIDList))
    #print(uniquePatientIDList)
    #count=0
    
    #for ID in uniquePatientIDList:
        #left_level=ImageLevelHash[str(ID+'_left')]
        #print(left_level)
        #right_level=ImageLevelHash[str(ID+'_right')]
        #if (left_level != right_level):
            #count = count+1
    #print("左右眼患病状态不同的病人有"+str(count)+"名")
    #print("共有"+str(len(uniquePatientIDList))+"名病人")
    #raw_df['level'].astype(np.int32)
    return raw_df

label=loadLabels()
keepImages =list(ImageNameDataHash.keys())
#print(keepImages)
print('完成')


共有413数据
拥有2特征
特征分为:['Image name', 'Retinopathy grade']
完成


In [3]:
image_arr=[]
data_arr=[]

for index, row in label.iterrows():
    key = str(row[0])
    if key in ImageNameDataHash:
        image_arr.append(key)
        data_arr.append(np.array(ImageNameDataHash[key]))
label2 = pd.DataFrame({'Image name': image_arr, 'data': data_arr})

label = pd.merge(label2, label, left_on='Image name', right_on='Image name', how='outer')
label_header_list = list(label.columns) 
print(label_header_list) 
print(len(label)) 
print(label.sample())

['Image name', 'data', 'Retinopathy grade']
413
    Image name                                               data  \
397  IDRiD_398  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   

     Retinopathy grade  
397                  2  


In [4]:
print("将训练集和测试集分为80%：20%")
sys.stdout.flush()
print("共有数据" + str(label['Image name'].nunique())+"张图片")
name = label['Image name'].unique()
#print('unique_ids shape='+ str(len(name)))

train_ids, valid_ids = train_test_split(name, test_size = 0.2, random_state = 10) 
name_list=name.tolist()
trainid_list = train_ids.tolist()
print('训练集有', str(len(trainid_list))+"张图片") 

traindf = label[label['Image name'].isin(name_list)]
valSet = label[~label['Image name'].isin(trainid_list)]
print("完成")

将训练集和测试集分为80%：20%
共有数据413张图片
训练集有 330张图片
完成


In [5]:
#print(traindf.head())
#print(valSet.head())

traindf = traindf.reset_index(drop=True)
valSet = valSet.reset_index(drop=True)

print(traindf.head())
print(valSet.head())
print("完成")

  Image name                                               data  \
0  IDRiD_001  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
1  IDRiD_002  [[[2.0, 2.0, 2.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
2  IDRiD_003  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
3  IDRiD_004  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [3.0, 0.0,...   
4  IDRiD_005  [[[0.0, 0.0, 0.0], [5.0, 1.0, 2.0], [8.75, 0.0...   

   Retinopathy grade  
0                  3  
1                  3  
2                  2  
3                  3  
4                  4  
  Image name                                               data  \
0  IDRiD_002  [[[2.0, 2.0, 2.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
1  IDRiD_007  [[[0.0, 0.0, 4.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
2  IDRiD_013  [[[5.0, 1.0, 2.0], [5.0, 1.0, 2.0], [5.0, 1.0,...   
3  IDRiD_025  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
4  IDRiD_027  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   

   Retinopathy grade  
0                  3  
1         

In [6]:
trainX = traindf['data']
trainY = traindf['Retinopathy grade']

valX = valSet['data']
valY = valSet['Retinopathy grade']

print('训练集为', trainX.shape[0], '验证集为', valX.shape[0])

训练集为 413 验证集为 83


In [7]:
trainY =  to_categorical(trainY, num_classes=5)
valY =  to_categorical(valY, num_classes=5)

print('完成')

完成


In [9]:
import tensorflow as tf



model_den=DenseNet121(include_top=False,weights=None,input_shape=(128,128,3))
x = model_den.output
x = GlobalAveragePooling2D()(x)
x= Dense(512, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model_final = Model(inputs=model_den.input, outputs=predictions)
model_final.summary()
model_final.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=[metrics.categorical_accuracy])
print("完成")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

__________________________________________________________________________________________________
conv4_block7_1_conv (Conv2D)    (None, 8, 8, 128)    57344       conv4_block7_0_relu[0][0]        
__________________________________________________________________________________________________
conv4_block7_1_bn (BatchNormali (None, 8, 8, 128)    512         conv4_block7_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block7_1_relu (Activation (None, 8, 8, 128)    0           conv4_block7_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block7_2_conv (Conv2D)    (None, 8, 8, 32)     36864       conv4_block7_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block7_concat (Concatenat (None, 8, 8, 480)    0           conv4_block6_concat[0][0]        
          

In [12]:
Xtrain = np.zeros([trainX.shape[0],128, 128,3])
for i in range(trainX.shape[0]):
    Xtrain[i] = trainX[i]
print(Xtrain.shape)

(413, 128, 128, 3)


In [13]:
Xval = np.zeros([valX.shape[0],128,128,3])
for i in range(valX.shape[0]):
    Xval[i] = valX[i]
print(Xval.shape)

(83, 128, 128, 3)


In [14]:
print("生成数据")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")
print("完成")

生成数据
完成


In [18]:
print("训练模型.")
sys.stdout.flush()

H = model_final.fit_generator(aug.flow(Xtrain, trainY, batch_size=10), \
    validation_data=(Xval, valY), \
    steps_per_epoch=len(trainX)//10, \
    
    epochs=50, verbose=1)


# save the model to disk
print("存储模型")
sys.stdout.flush()
model_final.save("dense.h5")
print("训练完成")

训练模型.
Epoch 1/50
41/41 [==============================] - 128s 3s/step - loss: 0.3468 - categorical_accuracy: 0.8707 - val_loss: 0.1730 - val_categorical_accuracy: 0.9639
Epoch 2/50
41/41 [==============================] - 129s 3s/step - loss: 0.2832 - categorical_accuracy: 0.8871 - val_loss: 0.2435 - val_categorical_accuracy: 0.8916
Epoch 3/50
41/41 [==============================] - 131s 3s/step - loss: 0.3205 - categorical_accuracy: 0.8926 - val_loss: 0.4499 - val_categorical_accuracy: 0.8916
Epoch 4/50
41/41 [==============================] - 130s 3s/step - loss: 0.2360 - categorical_accuracy: 0.9097 - val_loss: 0.5955 - val_categorical_accuracy: 0.8313
Epoch 5/50
41/41 [==============================] - 130s 3s/step - loss: 0.3868 - categorical_accuracy: 0.8846 - val_loss: 0.2327 - val_categorical_accuracy: 0.9277
Epoch 6/50
41/41 [==============================] - 129s 3s/step - loss: 0.3302 - categorical_accuracy: 0.8773 - val_loss: 0.9936 - val_categorical_accuracy: 0.8313
Epoc